# Data Scraper 
This notebook contains the code used to fill the `data/` folder. 

In [1]:
import requests 
import pandas as pd 
import os
from dotenv import load_dotenv
load_dotenv()


True

### Weather Data
The next set of cells will scrape weather data from X to Y and store it under `data/weather_data.csv`.

In [2]:
## Constants

BASE_WEATHER_API = "https://archive-api.open-meteo.com/v1/archive"
LATITUDE = 41.5868
LONGITUDE = 93.6250
START_DATE = "2000-03-24"
END_DATE = "2023-04-07"
DAILY = ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum"]
TIMEZONE = "GMT"
TEMPERATURE_UNITS = "celsius"
WINDSPEED_UNITS = "kmh"
PRECIPITATION_UNIT = "mm"
TIME_FORMAT = "iso8601"

In [3]:
query_params = f"latitude={LATITUDE}&longitude={LONGITUDE}&start_date={START_DATE}&end_date={END_DATE}&{'&'.join(f'daily={val}' for val in DAILY)}&timezone={TIMEZONE}&temperature_unit={TEMPERATURE_UNITS}&windspeed_unit={WINDSPEED_UNITS}&precipitation_unit={PRECIPITATION_UNIT}&timeformat={TIME_FORMAT}"
resp = requests.get(f"{BASE_WEATHER_API}?{query_params}")

json = resp.json() 

In [4]:
print(json["daily"].keys())

dict_keys(['time', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'precipitation_sum'])


As suggested by the code cell above, the API returned JSON with keys for each of the variables of interest; the values of each key are a list of values, one for each date. We now reformat the data to make it more usable. 

In [5]:
daily = json["daily"]

data = {}

for key in daily: 
    for i, val in enumerate(daily[key]): 
        if i not in data: data[i] = {}
        data[i][key] = val

# Showing results of transformation: 
for i in range(10): 
    print(data[i])

{'time': '2000-03-24', 'temperature_2m_max': 12.6, 'temperature_2m_min': -6.6, 'temperature_2m_mean': 5.7, 'precipitation_sum': 0.0}
{'time': '2000-03-25', 'temperature_2m_max': 15.0, 'temperature_2m_min': -1.0, 'temperature_2m_mean': 8.7, 'precipitation_sum': 0.0}
{'time': '2000-03-26', 'temperature_2m_max': 18.7, 'temperature_2m_min': 2.4, 'temperature_2m_mean': 11.2, 'precipitation_sum': 0.0}
{'time': '2000-03-27', 'temperature_2m_max': 22.9, 'temperature_2m_min': 1.5, 'temperature_2m_mean': 14.8, 'precipitation_sum': 0.0}
{'time': '2000-03-28', 'temperature_2m_max': 22.8, 'temperature_2m_min': 7.2, 'temperature_2m_mean': 14.8, 'precipitation_sum': 0.0}
{'time': '2000-03-29', 'temperature_2m_max': 20.1, 'temperature_2m_min': 6.2, 'temperature_2m_mean': 13.2, 'precipitation_sum': 0.0}
{'time': '2000-03-30', 'temperature_2m_max': 17.7, 'temperature_2m_min': 3.3, 'temperature_2m_mean': 11.8, 'precipitation_sum': 0.0}
{'time': '2000-03-31', 'temperature_2m_max': 14.7, 'temperature_2m_mi

In [6]:
weather_df = pd.DataFrame.from_dict(data=data, orient="index")
weather_df.tail()

,time,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum
8410,2023-04-03,6.0,-4.5,0.9,1.2
8411,2023-04-04,2.2,-6.6,-1.5,1.8
8412,2023-04-05,8.5,-7.0,2.3,0.0
8413,2023-04-06,12.1,-4.2,5.6,0.0
8414,2023-04-07,15.8,-0.5,9.8,0.0


In [7]:
weather_df.to_csv(path_or_buf="../data/weather_data.csv")

### Crop Data
The next set of cells will scrape crop price data from X to Y and store it under `data/crop_price_data.csv`.

In [8]:
## Constants

API_KEY = os.environ["CROP_DATA_API_KEY"]
BASE_CROP_PRICE_API = "https://financialmodelingprep.com/api/v3"
SYMBOL = "CUSX" # Corn future

In [9]:
# resp = requests.get(f"{BASE_CROP_PRICE_API}/historical-price-full/{SYMBOL}?apikey={API_KEY}")
# corn_json = resp.json()

In [27]:
def scrape_commodity_prices(symbol: str) -> pd.DataFrame: 
    resp = requests.get(f"{BASE_CROP_PRICE_API}/historical-price-full/{symbol}?apikey={API_KEY}")
    json = resp.json()
    
    data = {}
    for entry in json["historical"]: 
        data[entry["date"]] = {
            "open": entry["open"], 
            "close": entry["close"], 
            "high": entry["high"], 
            "low": entry["low"]
        }
    
    df = pd.DataFrame.from_dict(data=data, orient="index")
    
    return df



In [28]:
corn_df = scrape_commodity_prices(symbol="CUSX")
corn_df.head()

,open,close,high,low
2023-04-20,636.50,626.50,637.25,622.00
2023-04-19,675.75,672.25,680.00,671.00
2023-04-18,676.00,677.50,682.50,669.75
2023-04-17,666.00,676.50,677.00,662.25
2023-04-16,635.00,634.75,635.75,634.00


In [ ]:
corn_df.to_csv(path_or_buf="../data/corn_data.csv")

In [29]:
wheat_df = scrape_commodity_prices(symbol="KWUSX")
wheat_df.head()

,open,close,high,low
2023-04-20,850.75,827.25,852.25,822.50
2023-04-19,882.00,861.75,883.00,861.25
2023-04-18,887.50,881.25,898.75,876.00
2023-04-17,884.00,889.75,891.00,862.00
2023-04-16,884.00,862.75,884.00,875.25


In [30]:
wheat_df.to_csv(path_or_buf="../data/wheat_data.csv")

In [31]:
crude_oil_df = scrape_commodity_prices(symbol="CLUSD")
crude_oil_df.head()

,open,close,high,low
2023-04-20,78.870003,77.290001,78.889999,76.970001
2023-04-19,80.919998,79.160004,81.180000,78.459999
2023-04-18,81.000000,80.860001,81.480003,79.870003
2023-04-17,82.480003,80.830002,82.709999,80.470001
2023-04-16,82.480003,82.500000,82.680000,82.459999


In [32]:
crude_oil_df.to_csv(path_or_buf="../data/crude_oil.csv")